In [ ]:
!pip install qiskit-optimization
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    RecursiveMinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)
from qiskit_optimization import QuadraticProgram
from qiskit.visualization import plot_histogram
from typing import List, Tuple
import numpy as np


In [ ]:
# penalty constant
k = 10


In [ ]:
# create a QUBO
qubo = QuadraticProgram()
qubo.binary_var("x0")
qubo.binary_var("x1")
qubo.binary_var("x2")
qubo.binary_var("y0")
qubo.binary_var("y1")
qubo.binary_var("y2")
qubo.minimize(constant=81*k, linear=[-17*k,-32*k,-56*k,-17*k,-32*k,-56*k], quadratic={("x0","y0"):(2*k-1), ("x0","y1"):(4*k-2), ("x0","y2"):(8*k-4), ("x1","y0"):(4*k-2), ("x1","y1"):(8*k-4), ("x1","y2"):(16*k-8), ("x2","y0"):(8*k-4), ("x2","y1"):(16*k-8), ("x2","y2"):(32*k-16), ("x0","x1"):4*k, ("x0","x2"):8*k, ("x1","x2"):16*k, ("y0","y1"):4*k, ("y0","y2"):8*k, ("y1","y2"):16*k})
print(qubo.prettyprint())


In [ ]:
op, offset = qubo.to_ising()
print("offset: {}".format(offset))
print("operator:")
print(op)


In [ ]:
qp = QuadraticProgram()
qp.from_ising(op, offset, linear=True)
print(qp.prettyprint())


In [ ]:
algorithm_globals.random_seed = 10598
qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(), initial_point=[0.5, 0.5])
exact_mes = NumPyMinimumEigensolver()


In [ ]:
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver


In [ ]:
exact_result = exact.solve(qubo)
print(exact_result.prettyprint())


In [ ]:
qaoa_result = qaoa.solve(qubo)
print(qaoa_result.prettyprint())


In [ ]:
print("variable order:", [var.name for var in qaoa_result.variables])
for s in qaoa_result.samples:
    print(s)
